In [3]:
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt

In [ ]:
def nrrd_reader(file_path):
    _nrrd = nrrd.read(file_path)
    data = _nrrd[0]
    header = _nrrd[1]
    return data, header

def nibabel_reader(file_path):
    volume_nifty = nb.load(file_path)
    volume = volume_nifty.get_fdata()
    return volume, volume_nifty.header

def file_reader(file_path, file_type):
    header_mat = np.like((4,4))
    if file_type == 'nrrd':
        data, header = nrrd_reader(file_path)
        affine = header['space directions']
        affine = affine[:3, :3]
        origins = header['space origin']
        origins = origins[:3]
        t_mat = from_matvec(affine, origins)
        header_mat = t_mat
    else:
        data, header = nibabel_reader(file_path)
        print(header)
        header_mat = header
    
    return data, header_mat

In [ ]:
file_to_read = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_W_COMP_AD_-5_5.nii.gz'

data = nb.load(file_to_read)
print(data.header)
    

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 293 427 645   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.        1.6666666 1.6666666 1.7000046 0.        0.        0.
 0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : aligned
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.70710677
quatern_d       : -0.70710677
qoffset_x       : 266.92883
qoffset_y       : 99.03311
